<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>
<h1 align="center">Evaluate RAG with LLM Evals</h1>

In this tutorial we will look into building a RAG pipeline and evaluating it with Phoenix Evals.

It has the the following sections:

1. Understanding Retrieval Augmented Generation (RAG).
2. Building RAG (with the help of a framework such as LlamaIndex).
3. Evaluating RAG with Phoenix Evals.

## Retrieval Augmented Generation (RAG)

LLMs are trained on vast datasets, but these will not include your specific data (things like company knowledge bases and documentation). Retrieval-Augmented Generation (RAG) addresses this by dynamically incorporating your data as context during the generation process. This is done not by altering the training data of the LLMs but by allowing the model to access and utilize your data in real-time to provide more tailored and contextually relevant responses.

In RAG, your data is loaded and prepared for queries. This process is called indexing. User queries act on this index, which filters your data down to the most relevant context. This context and your query then are sent to the LLM along with a prompt, and the LLM provides a response.

RAG is a critical component for building applications such a chatbots or agents and you will want to know RAG techniques on how to get data into your application.

<img src="https://storage.googleapis.com/arize-phoenix-assets/assets/images/RAG_Pipeline.png">

## Stages within RAG

There are five key stages within RAG, which will in turn be a part of any larger RAG application.

- **Loading**: This refers to getting your data from where it lives - whether it's text files, PDFs, another website, a database or an API - into your pipeline.
- **Indexing**: This means creating a data structure that allows for querying the data. For LLMs this nearly always means creating vector embeddings, numerical representations of the meaning of your data, as well as numerous other metadata strategies to make it easy to accurately find contextually relevant data.
- **Storing**: Once your data is indexed, you will want to store your index, along with any other metadata, to avoid the need to re-index it.

- **Querying**: For any given indexing strategy there are many ways you can utilize LLMs and data structures to query, including sub-queries, multi-step queries, and hybrid strategies. 
- **Evaluation**: A critical step in any pipeline is checking how effective it is relative to other strategies, or when you make changes. Evaluation provides objective measures on how accurate, faithful, and fast your responses to queries are.


## Build a RAG system 

Now that we have understood the stages of RAG, let's build a pipeline. We will use [LlamaIndex](https://www.llamaindex.ai/) for RAG and [Phoenix Evals](https://docs.arize.com/phoenix/llm-evals/llm-evals) for evaluation.


In [1]:
!pip install -qq "arize-phoenix[experimental,llama-index]>=3.0.3"

In [2]:
# The nest_asyncio module enables the nesting of asynchronous functions within an already running async loop.
# This is necessary because Jupyter notebooks inherently operate in an asynchronous loop.
# By applying nest_asyncio, we can run additional async functions within this existing loop without conflicts.
import nest_asyncio

nest_asyncio.apply()

import os
from getpass import getpass

import pandas as pd
import phoenix as px
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, set_global_handler
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.llms.openai import OpenAI

/Users/mikeldking/work/phoenix/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/mikeldking/work/phoenix/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


During this tutorial, we will capture all the data we need to evaluate our RAG pipeline using Phoenix Tracing. To enable this, simply start the phoenix application and instrument LlamaIndex.

In [3]:
px.launch_app()

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [4]:
set_global_handler("arize_phoenix")

For this tutorial we will be using OpenAI for creating synthetic data as well as for evaluation. 

In [5]:
if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key

### Load Data and Build an Index

Let's use an [essay by Paul Graham](https://www.paulgraham.com/worked.html) to build our RAG pipeline.

In [6]:
import tempfile
from urllib.request import urlretrieve

with tempfile.NamedTemporaryFile() as tf:
    urlretrieve(
        "https://raw.githubusercontent.com/Arize-ai/phoenix-assets/main/data/paul_graham/paul_graham_essay.txt",
        tf.name,
    )
    documents = SimpleDirectoryReader(input_files=[tf.name]).load_data()

In [7]:
# Define an LLM
llm = OpenAI(model="gpt-4")

# Build index with a chunk_size of 512
node_parser = SimpleNodeParser.from_defaults(chunk_size=512)
nodes = node_parser.get_nodes_from_documents(documents)
vector_index = VectorStoreIndex(nodes)

Build a QueryEngine and start querying.

In [8]:
query_engine = vector_index.as_query_engine()

In [9]:
response_vector = query_engine.query("What did the author do growing up?")

Check the response that you get from the query.

In [10]:
response_vector.response

'The author mentioned that before college, they worked on writing and programming. They wrote short stories and also tried writing programs on the IBM 1401 computer. They used an early version of Fortran and had to type programs on punch cards. They also mentioned being puzzled by the 1401 and not being able to do much with it.'

By default LlamaIndex retrieves two similar nodes/ chunks. You can modify that in `vector_index.as_query_engine(similarity_top_k=k)`.

Let's check the text in each of these retrieved nodes.

In [11]:
# First retrieved node
response_vector.source_nodes[0].get_text()

'What I Worked On\n\nFebruary 2021\n\nBefore college the two main things I worked on, outside of school, were writing and programming. I didn\'t write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.\n\nThe first programs I tried writing were on the IBM 1401 that our school district used for what was then called "data processing." This was in 9th grade, so I was 13 or 14. The school district\'s 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and I got permission to use it. It was like a mini Bond villain\'s lair down there, with all these alien-looking machines — CPU, disk drives, printer, card reader — sitting up on a raised floor under bright fluorescent lights.\n\nThe language we used was an early version of Fortran. You had to type programs on punch cards, then stack

In [12]:
# Second retrieved node
response_vector.source_nodes[1].get_text()

"I also worked on spam filters, and did some more painting. I used to have dinners for a group of friends every thursday night, which taught me how to cook for groups. And I bought another building in Cambridge, a former candy factory (and later, twas said, porn studio), to use as an office.\n\nOne night in October 2003 there was a big party at my house. It was a clever idea of my friend Maria Daniels, who was one of the thursday diners. Three separate hosts would all invite their friends to one party. So for every guest, two thirds of the other guests would be people they didn't know but would probably like. One of the guests was someone I didn't know but would turn out to like a lot: a woman called Jessica Livingston. A couple days later I asked her out.\n\nJessica was in charge of marketing at a Boston investment bank. This bank thought it understood startups, but over the next year, as she met friends of mine from the startup world, she was surprised how different reality was. And 

Remember that we are using Phoenix Tracing to capture all the data we need to evaluate our RAG pipeline. You can view the traces in the phoenix application.

In [13]:
print("phoenix URL", px.active_session().url)

phoenix URL http://localhost:6006/


We can access the traces by directly pulling the spans from the phoenix session.

In [14]:
spans_df = px.Client().get_spans_dataframe()

In [15]:
spans_df[["name", "span_kind", "attributes.input.value", "attributes.retrieval.documents"]].head()

,name,span_kind,attributes.input.value,attributes.retrieval.documents
context.span_id,,,,
3f9ccdb4b6176bd1,llm,LLM,NaN,NaN
a1d3424b60e71f6e,chunking,CHAIN,NaN,NaN
010abeb51ba15621,chunking,CHAIN,NaN,NaN
adee02abd337a2bd,synthesize,CHAIN,What did the author do growing up?,NaN
01c5f9a72922607a,embedding,EMBEDDING,NaN,NaN


Note that the traces have captured the documents that were retrieved by the query engine. This is nice because it means we can introspect the documents without having to keep track of them ourselves.

In [16]:
spans_with_docs_df = spans_df[spans_df["attributes.retrieval.documents"].notnull()]

In [17]:
spans_with_docs_df[["attributes.input.value", "attributes.retrieval.documents"]].head()

,attributes.input.value,attributes.retrieval.documents
context.span_id,,
de62c77958d3ec4d,What did the author do growing up?,[{'document.metadata': {'file_path': '/var/fol...


We have built a RAG pipeline and also have instrumented it using Phoenix Tracing. We now need to evaluate it's performance. We can assess our RAG system/query engine using Phoenix's LLM Evals. Let's examine how to leverage these tools to quantify the quality of our retrieval-augmented generation system.

## Evaluation

Evaluation should serve as the primary metric for assessing your RAG application. It determines whether the pipeline will produce accurate responses based on the data sources and range of queries.

While it's beneficial to examine individual queries and responses, this approach is impractical as the volume of edge-cases and failures increases. Instead, it's more effective to establish a suite of metrics and automated evaluations. These tools can provide insights into overall system performance and can identify specific areas that may require scrutiny.

In a RAG system, evaluation focuses on two critical aspects:

- **Retrieval Evaluation**: To assess the accuracy and relevance of the documents that were retrieved
- **Response Evaluation**: Measure the appropriateness of the response generated by the system when the context was provided.

### Generate Question Context Pairs

For the evaluation of a RAG system, it's essential to have queries that can fetch the correct context and subsequently generate an appropriate response.

For this tutorial, let's use Phoenix's `llm_generate` to help us create the question-context pairs.

First, let's create a dataframe of all the document chunks that we have indexed.

In [18]:
# Let's construct a dataframe of just the documents that are in our index
document_chunks_df = pd.DataFrame({"text": [node.get_text() for node in nodes]})
document_chunks_df.head()

,text
0,What I Worked On\n\nFebruary 2021\n\nBefore co...
1,I was puzzled by the 1401. I couldn't figure o...
2,I remember vividly how impressed and envious I...
3,I couldn't have put this into words when I was...
4,The default language at Cornell was a Pascal-l...


Now that we have the document chunks, let's prompt an LLM to generate us 3 questions per chunk. Note that you could manually solicit questions from your team or customers, but this is a quick and easy way to generate a large number of questions.

In [19]:
generate_questions_template = """\
Context information is below.

---------------------
{text}
---------------------

Given the context information and not prior knowledge.
generate only questions based on the below query.

You are a Teacher/ Professor. Your task is to setup \
3 questions for an upcoming \
quiz/examination. The questions should be diverse in nature \
across the document. Restrict the questions to the \
context information provided."

Output the questions in JSON format with the keys question_1, question_2, question_3.
"""

In [20]:
import json

from phoenix.evals import OpenAIModel, llm_generate


def output_parser(response: str, index: int):
    try:
        return json.loads(response)
    except json.JSONDecodeError as e:
        return {"__error__": str(e)}


questions_df = llm_generate(
    dataframe=document_chunks_df,
    template=generate_questions_template,
    model=OpenAIModel(
        model_name="gpt-3.5-turbo",
    ),
    output_parser=output_parser,
    concurrency=20,
)

The `model_name` field is deprecated. Use `model` instead.                 This will be removed in a future release.


llm_generate |          | 0/61 (0.0%) | ⏳ 00:00<? | ?it/s

llm_generate |██████████| 61/61 (100.0%) | ⏳ 00:14<00:00 |  4.33it/s


In [21]:
questions_df.head()

,question_1,question_2,question_3
0,What were the two main things the author worke...,What was the language used for programming on ...,What was the author's clearest memory regardin...
1,What were the limitations of the 1401 computer...,How did microcomputers change the author's exp...,Why did the author's father buy a TRS-80 compu...
2,What was the author's first experience with co...,Why did the author decide to switch from study...,What were the two things that influenced the a...
3,What were the two things that inspired the aut...,What programming language did the author learn...,What was the author's undergraduate thesis about?
4,What was the default language at Cornell and o...,What did the author reverse-engineer for their...,What realization did the author have during th...


In [22]:
# Construct a dataframe of the questions and the document chunks
questions_with_document_chunk_df = pd.concat([questions_df, document_chunks_df], axis=1)
questions_with_document_chunk_df = questions_with_document_chunk_df.melt(
    id_vars=["text"], value_name="question"
).drop("variable", axis=1)
# If the above step was interrupted, there might be questions missing. Let's run this to clean up the dataframe.
questions_with_document_chunk_df = questions_with_document_chunk_df[
    questions_with_document_chunk_df["question"].notnull()
]

The LLM has generated three questions per chunk. Let's take a quick look.

In [23]:
questions_with_document_chunk_df.head(10)

,text,question
0,What I Worked On\n\nFebruary 2021\n\nBefore co...,What were the two main things the author worke...
1,I was puzzled by the 1401. I couldn't figure o...,What were the limitations of the 1401 computer...
2,I remember vividly how impressed and envious I...,What was the author's first experience with co...
3,I couldn't have put this into words when I was...,What were the two things that inspired the aut...
4,The default language at Cornell was a Pascal-l...,What was the default language at Cornell and o...
5,"I applied to 3 grad schools: MIT and Yale, whi...",What were the three grad schools that the auth...
6,So I looked around to see what I could salvage...,What was the main reason people cared about Li...
7,"And indeed, it would seem very feeble work. On...",What realization did the author have while loo...
8,And as an artist you could be truly independen...,What was the author's initial perception of ar...
9,I remember when my friend Robert Morris got ki...,What was the reason for Robert Morris getting ...


### Retrieval Evaluation

We are now prepared to perform our retrieval evaluations. We will execute the queries we generated in the previous step and verify whether or not that the correct context is retrieved.

In [24]:
# First things first, let's reset phoenix
px.close_app()
px.launch_app()

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [25]:
# loop over the questions and generate the answers
for _, row in questions_with_document_chunk_df.iterrows():
    question = row["question"]
    response_vector = query_engine.query(question)
    print(f"Question: {question}\nAnswer: {response_vector.response}\n")

Question: What were the two main things the author worked on before college?
Answer: Before college, the author worked on writing and programming.

Question: What were the limitations of the 1401 computer and why couldn't the author do much with it?
Answer: The 1401 computer had limitations in terms of input. The only form of input to programs was data stored on punched cards, and the author did not have any data stored on punched cards. Additionally, the author did not know enough math to do anything interesting that didn't rely on input. Therefore, the author couldn't do much with the 1401 computer due to these limitations.

Question: What was the author's first experience with computers and programming?
Answer: The author's first experience with computers and programming was in 9th grade when they had access to an IBM 1401 computer in the basement of their junior high school. They used an early version of Fortran and had to type programs on punch cards. However, they were puzzled by

Now that we have executed the queries, we can start validating whether or not the RAG system was able to retrieve the correct context. Let's extract all the retrieved documents from the traces logged to phoenix. (For an in-depth explanation of how to export trace data from the phoenix runtime, consult the [docs](https://docs.arize.com/phoenix/how-to/extract-data-from-spans)).

In [26]:
from phoenix.session.evaluation import get_retrieved_documents

retrieved_documents_df = get_retrieved_documents(px.Client())
retrieved_documents_df

context.trace_id  \
context.span_id  document_position                                     
10a431236f055f02 0                  8f431d484026a4a49d49f61704e17900   
                 1                  8f431d484026a4a49d49f61704e17900   
dc793d36a608ba5c 0                  a3fd4f055c7f57a0a4c9cc295f1f5de9   
                 1                  a3fd4f055c7f57a0a4c9cc295f1f5de9   
aa3d3b3c376d1f05 0                  930c594dde71a6c8d7343042cdf0307d   
...                                                              ...   
09293733c663d225 1                  efc4aff9f4676fb6d049a5a45d1e9826   
2b06b57e15ece6c1 0                  624d610a75d4e5dd81c03f006bc6d551   
                 1                  624d610a75d4e5dd81c03f006bc6d551   
e0c4b7a59d9ccc43 0                  a535c73074a829fe669e744b3bfac020   
                 1                  a535c73074a829fe669e744b3bfac020   

                                                                                input  \
context.span_id  document_position                                                      
10a431236f055f02 0                  Who are the individuals mentioned in the ackno...   
                 1                  Who are the individuals mentioned in the ackno...   
dc793d36a608ba5c 0                  What was the worst thing about leaving YC acco...   
                 1                  What was the worst thing about leaving YC acco...   
aa3d3b3c376d1f05 0                   Why did YC choose the color orange for its logo?   
...                                                                               ...   
09293733c663d225 1                  What was the author's first experience with co...   
2b06b57e15ece6c1 0                  What were the limitations of the 1401 computer...   
                 1                  What were the limitations of the 1401 computer...   
e0c4b7a59d9ccc43 0                  What were the two main things the author worke...   
                 1                  What were the two main things the author worke...   

                                                                            reference  \
context.span_id  document_position                                                      
10a431236f055f02 0                  I also worked on spam filters, and did some mo...   
                 1                  Individually these two phenomena are tedious b...   
dc793d36a608ba5c 0                  Surely the biggest source of stress in one's w...   
                 1                  If you were trying to learn the most you could...   
aa3d3b3c376d1f05 0                  Customary VC practice had once, like the custo...   
...                                                                               ...   
09293733c663d225 1                  What I Worked On\n\nFebruary 2021\n\nBefore co...   
2b06b57e15ece6c1 0                  I was puzzled by the 1401. I couldn't figure o...   
                 1                  What I Worked On\n\nFebruary 2021\n\nBefore co...   
e0c4b7a59d9ccc43 0                  What I Worked On\n\nFebruary 2021\n\nBefore co...   
                 1                  Even then it took me several years to understa...   

                                    document_score  
context.span_id  document_position                  
10a431236f055f02 0                        0.745132  
                 1                        0.744114  
dc793d36a608ba5c 0                        0.844394  
                 1                        0.834679  
aa3d3b3c376d1f05 0                        0.845311  
...                                            ...  
09293733c663d225 1                        0.879562  
2b06b57e15ece6c1 0                        0.854816  
                 1                        0.838657  
e0c4b7a59d9ccc43 0                        0.839546  
                 1                        0.815409  

[366 rows x 4 columns]

Let's now use Phoenix's LLM Evals to evaluate the relevance of the retrieved documents with regards to the query. Note, we've turned on `explanations` which prompts the LLM to explain it's reasoning. This can be useful for debugging and for figuring out potential corrective actions.

In [27]:
from phoenix.evals import (
    RelevanceEvaluator,
    run_evals,
)

relevance_evaluator = RelevanceEvaluator(OpenAIModel(model="gpt-4-turbo-preview"))

retrieved_documents_relevance_df = run_evals(
    evaluators=[relevance_evaluator],
    dataframe=retrieved_documents_df,
    provide_explanation=True,
    concurrency=20,
)[0]

The `model_name` field is deprecated. Use `model` instead.                 This will be removed in a future release.


run_evals |          | 0/366 (0.0%) | ⏳ 00:00<? | ?it/s

run_evals |██████████| 366/366 (100.0%) | ⏳ 02:42<00:00 |  2.25it/s


In [28]:
retrieved_documents_relevance_df.head()

label  score  \
context.span_id  document_position                    
10a431236f055f02 0                  relevant      1   
                 1                  relevant      1   
dc793d36a608ba5c 0                  relevant      1   
                 1                  relevant      1   
aa3d3b3c376d1f05 0                  relevant      1   

                                                                          explanation  
context.span_id  document_position                                                     
10a431236f055f02 0                  The question asks for the individuals mentione...  
                 1                  The question asks for the names of individuals...  
dc793d36a608ba5c 0                  The reference text provides detailed insights ...  
                 1                  The question asks for the worst thing about le...  
aa3d3b3c376d1f05 0                  The question asks why Y Combinator (YC) chose ...

We can now combine the documents with the relevance evaluations to compute retrieval metrics. These metrics will help us understand how well the RAG system is performing.

In [29]:
documents_with_relevance_df = pd.concat(
    [retrieved_documents_df, retrieved_documents_relevance_df.add_prefix("eval_")], axis=1
)
documents_with_relevance_df

context.trace_id  \
context.span_id  document_position                                     
10a431236f055f02 0                  8f431d484026a4a49d49f61704e17900   
                 1                  8f431d484026a4a49d49f61704e17900   
dc793d36a608ba5c 0                  a3fd4f055c7f57a0a4c9cc295f1f5de9   
                 1                  a3fd4f055c7f57a0a4c9cc295f1f5de9   
aa3d3b3c376d1f05 0                  930c594dde71a6c8d7343042cdf0307d   
...                                                              ...   
09293733c663d225 1                  efc4aff9f4676fb6d049a5a45d1e9826   
2b06b57e15ece6c1 0                  624d610a75d4e5dd81c03f006bc6d551   
                 1                  624d610a75d4e5dd81c03f006bc6d551   
e0c4b7a59d9ccc43 0                  a535c73074a829fe669e744b3bfac020   
                 1                  a535c73074a829fe669e744b3bfac020   

                                                                                input  \
context.span_id  document_position                                                      
10a431236f055f02 0                  Who are the individuals mentioned in the ackno...   
                 1                  Who are the individuals mentioned in the ackno...   
dc793d36a608ba5c 0                  What was the worst thing about leaving YC acco...   
                 1                  What was the worst thing about leaving YC acco...   
aa3d3b3c376d1f05 0                   Why did YC choose the color orange for its logo?   
...                                                                               ...   
09293733c663d225 1                  What was the author's first experience with co...   
2b06b57e15ece6c1 0                  What were the limitations of the 1401 computer...   
                 1                  What were the limitations of the 1401 computer...   
e0c4b7a59d9ccc43 0                  What were the two main things the author worke...   
                 1                  What were the two main things the author worke...   

                                                                            reference  \
context.span_id  document_position                                                      
10a431236f055f02 0                  I also worked on spam filters, and did some mo...   
                 1                  Individually these two phenomena are tedious b...   
dc793d36a608ba5c 0                  Surely the biggest source of stress in one's w...   
                 1                  If you were trying to learn the most you could...   
aa3d3b3c376d1f05 0                  Customary VC practice had once, like the custo...   
...                                                                               ...   
09293733c663d225 1                  What I Worked On\n\nFebruary 2021\n\nBefore co...   
2b06b57e15ece6c1 0                  I was puzzled by the 1401. I couldn't figure o...   
                 1                  What I Worked On\n\nFebruary 2021\n\nBefore co...   
e0c4b7a59d9ccc43 0                  What I Worked On\n\nFebruary 2021\n\nBefore co...   
                 1                  Even then it took me several years to understa...   

                                    document_score eval_label  eval_score  \
context.span_id  document_position                                          
10a431236f055f02 0                        0.745132   relevant           1   
                 1                        0.744114   relevant           1   
dc793d36a608ba5c 0                        0.844394   relevant           1   
                 1                        0.834679   relevant           1   
aa3d3b3c376d1f05 0                        0.845311   relevant           1   
...                                            ...        ...         ...   
09293733c663d225 1                        0.879562   relevant           1   
2b06b57e15ece6c1 0                        0.854816   relevant           1   
                 1           

Let's compute Normalized Discounted Cumulative Gain [NCDG](https://en.wikipedia.org/wiki/Discounted_cumulative_gain) at 2 for all our retrieval steps.  In information retrieval, this metric is often used to measure effectiveness of search engine algorithms and related applications.

In [30]:
import numpy as np
from sklearn.metrics import ndcg_score


def _compute_ndcg(df: pd.DataFrame, k: int):
    """Compute NDCG@k in the presence of missing values"""
    n = max(2, len(df))
    eval_scores = np.zeros(n)
    doc_scores = np.zeros(n)
    eval_scores[: len(df)] = df.eval_score
    doc_scores[: len(df)] = df.document_score
    try:
        return ndcg_score([eval_scores], [doc_scores], k=k)
    except ValueError:
        return np.nan


ndcg_at_2 = pd.DataFrame(
    {"score": documents_with_relevance_df.groupby("context.span_id").apply(_compute_ndcg, k=2)}
)

In [31]:
ndcg_at_2

,score
context.span_id,
006cb7f220df32bf,1.0
0149804012ae254c,1.0
023137a6f8f071b3,1.0
033ed01b4a4282c1,1.0
053211ee43444c44,1.0
...,...
f5b40573fd43f091,0.0
f76aa6f37562d097,1.0
f80fc1c11f813e14,1.0


Let's also compute precision at 2 for all our retrieval steps.

In [32]:
precision_at_2 = pd.DataFrame(
    {
        "score": documents_with_relevance_df.groupby("context.span_id").apply(
            lambda x: x.eval_score[:2].sum(skipna=False) / 2
        )
    }
)

In [33]:
precision_at_2

,score
context.span_id,
006cb7f220df32bf,1.0
0149804012ae254c,1.0
023137a6f8f071b3,1.0
033ed01b4a4282c1,1.0
053211ee43444c44,1.0
...,...
f5b40573fd43f091,0.0
f76aa6f37562d097,1.0
f80fc1c11f813e14,1.0


Lastly, let's compute whether or not a correct document was retrieved at all for each query (e.g. a hit)

In [34]:
hit = pd.DataFrame(
    {
        "hit": documents_with_relevance_df.groupby("context.span_id").apply(
            lambda x: x.eval_score[:2].sum(skipna=False) > 0
        )
    }
)

Let's now view the results in a combined dataframe.

In [35]:
retrievals_df = px.Client().get_spans_dataframe("span_kind == 'RETRIEVER'")
rag_evaluation_dataframe = pd.concat(
    [
        retrievals_df["attributes.input.value"],
        ndcg_at_2.add_prefix("ncdg@2_"),
        precision_at_2.add_prefix("precision@2_"),
        hit,
    ],
    axis=1,
)
rag_evaluation_dataframe

,attributes.input.value,ncdg@2_score,precision@2_score,hit
context.span_id,,,,
10a431236f055f02,Who are the individuals mentioned in the ackno...,1.00000,1.0,True
dc793d36a608ba5c,What was the worst thing about leaving YC acco...,1.00000,1.0,True
aa3d3b3c376d1f05,Why did YC choose the color orange for its logo?,1.00000,1.0,True
72ace962ee850df6,What is the lesson learned from the experience...,1.00000,1.0,True
357e38a97d2f621e,Why did the author have to launch the online s...,0.00000,0.0,False
...,...,...,...,...
14d908407c3b6d6d,What was the default language at Cornell and o...,1.00000,0.5,True
bc7353b79dfcd121,What were the two things that inspired the aut...,0.63093,0.5,True
09293733c663d225,What was the author's first experience with co...,1.00000,1.0,True


### Observations

Let's now take our results and aggregate them to get a sense of how well our RAG system is performing.

In [36]:
# Aggregate the scores across the retrievals
results = rag_evaluation_dataframe.mean(numeric_only=True)
results

ncdg@2_score         0.903329
precision@2_score    0.789617
hit                  0.923497
dtype: float64

As we can see from the above numbers, our RAG system is not perfect, there are times when it fails to retrieve the correct context within the first two documents. At other times the correct context is included in the top 2 results but non-relevant information is also included in the context. This is an indication that we need to improve our retrieval strategy. One possible solution could be to increase the number of documents retrieved and then use a more sophisticated ranking strategy (such as a reranker) to select the correct context.

We have now evaluated our RAG system's retrieval performance. Let's send these evaluations to Phoenix for visualization. By sending the evaluations to Phoenix, you will be able to view the evaluations alongside the traces that were captured earlier.

In [37]:
from phoenix.trace import DocumentEvaluations, SpanEvaluations

px.Client().log_evaluations(
    SpanEvaluations(dataframe=ndcg_at_2, eval_name="ndcg@2"),
    SpanEvaluations(dataframe=precision_at_2, eval_name="precision@2"),
    DocumentEvaluations(dataframe=retrieved_documents_relevance_df, eval_name="relevance"),
)

### Response Evaluation

The retrieval evaluations demonstrates that our RAG system is not perfect. However, it's possible that the LLM is able to generate the correct response even when the context is incorrect. Let's evaluate the responses generated by the LLM.

In [38]:
from phoenix.session.evaluation import get_qa_with_reference

qa_with_reference_df = get_qa_with_reference(px.Client())
qa_with_reference_df

,input,output,reference
context.span_id,,,
3a5fab8dc2c8a8f3,Who are the individuals mentioned in the ackno...,The individuals mentioned in the acknowledgmen...,"I also worked on spam filters, and did some mo..."
025c1fe949b6b35c,What was the worst thing about leaving YC acco...,The author does not mention the worst thing ab...,Surely the biggest source of stress in one's w...
3d93432bb1b09b5f,Why did YC choose the color orange for its logo?,YC chose the color orange for its logo because...,"Customary VC practice had once, like the custo..."
9d4270b647541677,What is the lesson learned from the experience...,Independent-minded individuals have an advanta...,"Customary VC practice had once, like the custo..."
03739c75cef7ede0,Why did the author have to launch the online s...,The author had to launch the online store buil...,Which meant being easy to use and inexpensive....
...,...,...,...
ef6728cb089ffa1e,What was the default language at Cornell and o...,The default language at Cornell and other univ...,The default language at Cornell was a Pascal-l...
d1f53885e4e005c5,What were the two things that inspired the aut...,The two things that inspired the author to wor...,"I applied to 3 grad schools: MIT and Yale, whi..."
b6504b6e102183ad,What was the author's first experience with co...,The author's first experience with computers a...,I remember vividly how impressed and envious I...


Now that we have a dataset of the question, context, and response (input, reference, and output), we now can measure how well the LLM is responding to the queries. For details on the QA correctness evaluation, see the [LLM Evals documentation](https://docs.arize.com/phoenix/llm-evals/running-pre-tested-evals/q-and-a-on-retrieved-data).

In [39]:
from phoenix.evals import (
    HallucinationEvaluator,
    OpenAIModel,
    QAEvaluator,
    run_evals,
)

qa_evaluator = QAEvaluator(OpenAIModel(model="gpt-4-turbo-preview"))
hallucination_evaluator = HallucinationEvaluator(OpenAIModel(model="gpt-4-turbo-preview"))

qa_correctness_eval_df, hallucination_eval_df = run_evals(
    evaluators=[qa_evaluator, hallucination_evaluator],
    dataframe=qa_with_reference_df,
    provide_explanation=True,
    concurrency=20,
)

run_evals |          | 0/366 (0.0%) | ⏳ 00:00<? | ?it/s

run_evals |██████████| 366/366 (100.0%) | ⏳ 03:20<00:00 |  1.82it/s


In [40]:
qa_correctness_eval_df.head()

,label,score,explanation
context.span_id,,,
3a5fab8dc2c8a8f3,correct,1,"To determine if the answer is correct, we need..."
025c1fe949b6b35c,correct,1,The question asks about the worst thing the au...
3d93432bb1b09b5f,correct,1,The reference text provides a clear explanatio...
9d4270b647541677,correct,1,The answer correctly identifies the lesson lea...
03739c75cef7ede0,incorrect,0,The reference text does not explicitly state t...


In [41]:
hallucination_eval_df.head()

,label,score,explanation
context.span_id,,,
3a5fab8dc2c8a8f3,factual,0,To determine if the answer is factual or hallu...
025c1fe949b6b35c,factual,0,The reference text does not explicitly state w...
3d93432bb1b09b5f,factual,0,The answer provided is factual based on the re...
9d4270b647541677,factual,0,The answer accurately reflects the content and...
03739c75cef7ede0,hallucinated,1,The reference text does mention that the compa...


#### Observations

Let's now take our results and aggregate them to get a sense of how well the LLM is answering the questions given the context.

In [42]:
qa_correctness_eval_df.mean(numeric_only=True)

score    0.857923
dtype: float64

In [43]:
hallucination_eval_df.mean(numeric_only=True)

score    0.098361
dtype: float64

Our QA Correctness score of `0.91` and a Hallucinations score `0.05` signifies that the generated answers are correct ~91% of the time and that the responses contain hallucinations 5% of the time - there is room for improvement. This could be due to the retrieval strategy or the LLM itself. We will need to investigate further to determine the root cause.

Since we have evaluated our RAG system's QA performance and Hallucinations performance, let's send these evaluations to Phoenix for visualization.

In [44]:
from phoenix.trace import SpanEvaluations

px.Client().log_evaluations(
    SpanEvaluations(dataframe=qa_correctness_eval_df, eval_name="Q&A Correctness"),
    SpanEvaluations(dataframe=hallucination_eval_df, eval_name="Hallucination"),
)

We now have sent all our evaluations to Phoenix. Let's go to the Phoenix application and view the results! Since we've sent all the evals to Phoenix, we can analyze the results together to make a determination on whether or not poor retrieval or irrelevant context has an effect on the LLM's ability to generate the correct response.

In [45]:
print("phoenix URL", px.active_session().url)

phoenix URL http://localhost:6006/


## Conclusion

We have explored how to build and evaluate a RAG pipeline using LlamaIndex and Phoenix, with a specific focus on evaluating the retrieval system and generated responses within the pipelines. 

Phoenix offers a variety of other evaluations that can be used to assess the performance of your LLM Application. For more details, see the [LLM Evals](https://docs.arize.com/phoenix/llm-evals/llm-evals) documentation.